# Raster data

[GDAL](https://gdal.org/) is today the reference toolbox to read and write raster data. It is used by almost all of the FOSS4G programmes and libraries that interact with rasters. GDAL is also used by many commercial products. A [Python API](https://gdal.org/python/) is available for GDAL that provides much of the functionality.

The [RasterIO](https://rasterio.readthedocs.io) library makes interaction with rasters considebrably more convinient, however. It is in esssence a bridge between GDAL and the [NumPy](https://numpy.org/) package for scientific computing. With RasterIO rasters are easily translated into NumPy arrays and vice-versa.

## Reading in and inspecting a raster

Possibly the most essential operation is to open a raster for processing or inspection. This is rather simple:



In [ ]:
import rasterio
srtm = rasterio.open('../data/srtm/srtm.geotiff')
srtm

This is a tile from the digital elevation model produced by the Shuttle Radio Topography Mission. 

The `open` method returns an object of the class `DatasetReader`, which contains the raster meta-data and the set of bands included. `open` can also be invoked in write (with the extra argument `'w'`, in which case it returns a `DatasetWriter` type object.

`DatasetReader` provides easy access to useful meta-data, for instance its dimensions:

In [ ]:
srtm.width

In [ ]:
srtm.height

Also information related to the reader object itself:

In [ ]:
srtm.name

In [ ]:
srtm.mode

In [ ]:
The `count` property informs on the number of bands:

In [ ]:
srtm.count

In [ ]:
The geographic information is also available:

In [ ]:
srtm.crs

In [ ]:
srtm.bounds

The `read` method provides the means to actually access the information in the raster. It takes as argument the band number and returns a NumPy [NDArray](https://docs.scipy.org/doc/numpy/reference/generated/numpy.ndarray.html) object:

In [ ]:
srtm_array = srtm.read(1)
srtm_array

## Plotting a raster

With the NumPy Array object plotting becomes rather simple. The [matplotlib](https://matplotlib.org) library is meant for Maths but it works rather smootly with rasters too.

In [ ]:
from matplotlib import pyplot
pyplot.imshow(srtm_array, cmap='pink')
pyplot.show()

This is a very simply form of visualisation. It does not allow image composition with different bands, for instance, but it is handy for quick visual inspection. The [Visualisation](07-visualization.ipynb) section explores this topic in more detail.

## Creating a new raster

With a raster band encapsulated into an array, map algebra and analysis in general become rather simple. As an example, consider the high pass filter defined by the moving window below:

|  0 | -1 |  0 |
| -1 |  4 | -1 |
|  0 | -1 |  0 |

This is a typical operation in digital imaging processing that also finds application in raster analysis. Applying this moving window to a NumPy array requiresthe following:

In [ ]:
import numpy as np
high_pass_array = np.full(srtm_array.shape, 0).astype(np.int16)

for i in range (1, srtm_array.shape[0] - 1):
    for j in range (1, srtm_array.shape[1] - 1):
        high_pass_array[i,j] = \
            srtm_array[i,j] * 4 - \
            srtm_array[i-1,j] - \
            srtm_array[i,j+1] - \
            srtm_array[i,j-1] - \
            srtm_array[i+1,j]
            
high_pass_array

In [ ]:
`matplotlib` can be applied again to inspect the result:

In [ ]:
pyplot.imshow(high_pass_array, cmap='pink')
pyplot.show()

The high pass filter highlights transitions (the darker cells in the raster above). Since the input is a DEM, in this case the high pass filter shows the cells with steepest slope.

This new raster is still just an array. In order to save it (or use it in general) as a geo-referenced object, a new dataset object must be created. This can be made with the `open` method in write mode, i.e. using the `'w'` argument:

In [ ]:
high_pass = rasterio.open(
    '../data/srtm/srtm_high_pass.geotiff',
    'w',
    driver='GTiff',
    height=high_pass_array.shape[0],
    width=high_pass_array.shape[1],
    count=1,
    dtype=str(high_pass_array.dtype),
    crs=srtm.crs,
    transform=srtm.transform
)

In write mode `open` needs some more information. The number of rows and columns, plus the data type, are passed from the array object itself. The CRS information is obtained from the original `srtm` object. Only the content of the cells in the new raster is different, the location and size remain the same. 

`open` created a connection to a new raster file and returned an object of type `RasterUpdater`. With this object the contents of the new array can be copied into the file:


In [ ]:
high_pass.write(high_pass_array, 1)

The `write` method takes as arguments the NumPy array with cell contents and the band number in the raster. In this case `1` is used for the band number, as there is only one.

The flush the file into the hard disk the `close` method is used:

In [ ]:
high_pass.close()

---
[<- Vector data](04-vector-data.ipynb) | [Data analysis ->](06-data-analysis.ipynb)